<a id="top"></a>
# Überschrift

<a href="#datenaufbereitung">1. Datenaufbereitung & Filtern der geretweeteten Accountnamen</a> </br>
<a href="#zaehlen-retweets">2. Zählen der Retweets in einem Testdatensatz</a> </br>
<a href="#komplett">3. Kompletter Datensatz</a>



<a id="datenaufbereitung"></a>
## 1. Datenaufbereitung & Filtern der geretweeteten Accountnamen

Zunächst werden aus dem Datensatz alle Accountnamen gefiltert, die retweeted wurden.

In [1]:
import pandas as pd
import re

# Datenaufbereitung: Reduzieren der Daten auf die Twitter-Handles

# 1. Daten einlesen
trump_twitter = pd.read_csv("data/trump_twitter_2009-2020.csv", sep=",")

In [2]:
# 2. Filtern der Zeilen mit Retweets (is_retweet == True)
retweets = trump_twitter[trump_twitter["isRetweet"] == "t"]

In [3]:
# 3. Drop aller Spalten ausser dem Tweet-Text und Reindexing00 der Spalten:
handles = retweets.drop(["id", "isRetweet", "isDeleted", "device", "favorites", "retweets", "date"], axis=1)
handles = handles.reset_index()
handles = handles.drop("index", axis=1)

In [4]:
# 4. Herausfiltern der Handles
retweeted_accounts = []

for index, row in handles.iterrows():
    # print(re.search(r'(@)\w+', str(row)).group())
    account = re.search(r'(@)\w+', str(row)).group()
    retweeted_accounts.append(account)

len(retweeted_accounts)

9335

<a href="#top">$\uparrow$ Top</a>

<a id="zaehlen-retweets"></a>
## 2. Zählen der Retweets in einem Testdatensatz

**Filter der Testdaten**: Erstellen eines Testdatensatzes mit den ersten 20 Account, die geretweeted wurden

In [5]:
testdata = retweeted_accounts[:20]
testdata

['@CBS_Herridge',
 '@MZHemingway',
 '@WhiteHouse',
 '@EricTrump',
 '@GOPChairwoman',
 '@TomFitton',
 '@marklevinshow',
 '@RealRLimbaugh',
 '@TheRightMelissa',
 '@megynkelly',
 '@DanScavino',
 '@chefjclark',
 '@TeamTrump',
 '@NRA',
 '@dbongino',
 '@dbongino',
 '@HeyTammyBruce',
 '@HeyTammyBruce',
 '@LeeSmithDC',
 '@ArthurSchwartz']

**Zählen der Anzahl Retweets**: Dieser Loop prüft, ob ein Account-Name im Dictionary in der Liste des keys `account_name` bereits vorhanden ist. Ist das nicht der Fall wird der Account der Liste hinzugefügt und der Liste des Keys `retweet_count` der Wert 1. Ist der Name bereits in `account_name` vorhanden, wird die Zahl an der entsprechenden Stelle der Liste `retweet_count` um 1 erhöht. 

In [9]:
retweet_counter = {"account_name": [], "retweet_count": []}

for account in testdata:
    if account not in retweet_counter["account_name"]:
        retweet_counter["account_name"].append(account)
        retweet_counter["retweet_count"].append(1)
    else:
        account_position = retweet_counter["account_name"].index(account)
        retweet_counter["retweet_count"][account_position] += 1
        
retweet_counter

{'account_name': ['@CBS_Herridge',
  '@MZHemingway',
  '@WhiteHouse',
  '@EricTrump',
  '@GOPChairwoman',
  '@TomFitton',
  '@marklevinshow',
  '@RealRLimbaugh',
  '@TheRightMelissa',
  '@megynkelly',
  '@DanScavino',
  '@chefjclark',
  '@TeamTrump',
  '@NRA',
  '@dbongino',
  '@HeyTammyBruce',
  '@LeeSmithDC',
  '@ArthurSchwartz'],
 'retweet_count': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1]}

**Prüfen, ob das funktioniert**: Beide Listen (`account_name` und `retweet_count` sollten 18 Elemente enthalten, da 2 Accounts 2 mal geretweetet wurden.

In [10]:
print(len(retweet_counter["account_name"]))
print(len(retweet_counter["retweet_count"]))

18
18


Zusammenführen der beiden Listen im Dictionary oben in einzelne Dictionaries für jeden Account $\rightarrow$ damit ist klar, welche key-value-Paare zusammengehören und wie oft ein Account retweeted wurde, ausserdem können die dictionaries einfach sortiert werden.

In [33]:
number_of_retweets = {retweet_counter["account_name"][i]: retweet_counter["retweet_count"][i] for i in range(len(retweet_counter["account_name"]))}
number_of_retweets

{'@CBS_Herridge': 1,
 '@MZHemingway': 1,
 '@WhiteHouse': 1,
 '@EricTrump': 1,
 '@GOPChairwoman': 1,
 '@TomFitton': 1,
 '@marklevinshow': 1,
 '@RealRLimbaugh': 1,
 '@TheRightMelissa': 1,
 '@megynkelly': 1,
 '@DanScavino': 1,
 '@chefjclark': 1,
 '@TeamTrump': 1,
 '@NRA': 1,
 '@dbongino': 2,
 '@HeyTammyBruce': 2,
 '@LeeSmithDC': 1,
 '@ArthurSchwartz': 1}

**Absteigend sortieren nach Anzahl Retweets:**

In [38]:
sorted_rt = sorted(number_of_retweets.items(), key = lambda kv: kv[1], reverse = True)
sorted_rt

[('@dbongino', 2),
 ('@HeyTammyBruce', 2),
 ('@CBS_Herridge', 1),
 ('@MZHemingway', 1),
 ('@WhiteHouse', 1),
 ('@EricTrump', 1),
 ('@GOPChairwoman', 1),
 ('@TomFitton', 1),
 ('@marklevinshow', 1),
 ('@RealRLimbaugh', 1),
 ('@TheRightMelissa', 1),
 ('@megynkelly', 1),
 ('@DanScavino', 1),
 ('@chefjclark', 1),
 ('@TeamTrump', 1),
 ('@NRA', 1),
 ('@LeeSmithDC', 1),
 ('@ArthurSchwartz', 1)]

In [37]:
len(sorted_rt)

18

<a href="#top">$\uparrow$ Top</a>

<a id="komplett"></a>
## 3. Kompletter Datensatz

Hier ist alles in einem Codeblock zusammengefasst und die Datenaufbereitung/Filterung und das Zusammenfassen der Accounts und Retweets ist in Funktionen verpackt.

In [64]:
import pandas as pd
import re


# Read data:
data = pd.read_csv("data/trump_twitter_2009-2020.csv", sep=",")


def get_retweeted_accounts(trump_twitter):
    """Returns a list of every twitter account DT ever retweeted"""
    
    # Filter rows with retweets only:
    retweets = trump_twitter[trump_twitter["isRetweet"] == "t"]
    
    # Drop all columns except "text" and reindex column:
    handles = retweets.drop(["id", "isRetweet", "isDeleted", "device", "favorites", "retweets", "date"], axis=1)
    handles = handles.reset_index()
    handles = handles.drop("index", axis=1)
    
    # Filter the account handles with regex:
    retweeted_accounts = []

    for index, row in handles.iterrows():
        account = re.search(r'(@)\w+', str(row)).group()
        retweeted_accounts.append(account)
    
    return retweeted_accounts


def get_number_of_retweets(tweets):
    """Returns a list of accounts, sorted by number of retweets."""
    
    # Empty dicts, later used for list concatenation:
    retweet_counter = {"account_name": [], "retweet_count": []}
    
    # Fill the lists in the dict with the respective values:
    for account in tweets:
        if account not in retweet_counter["account_name"]:
            retweet_counter["account_name"].append(account)
            retweet_counter["retweet_count"].append(1)
        else:
            account_position = retweet_counter["account_name"].index(account)
            retweet_counter["retweet_count"][account_position] += 1
    
    # Concatenate the lists in the retweet_counter dict, return single dictionary:
    retweets_pairing = {retweet_counter["account_name"][i]:
                        retweet_counter["retweet_count"][i]
                        for i in range(len(retweet_counter["account_name"]))}
    
    # Sort the key-value-pairs in the retweets_pairing dictionary in descending order: 
    retweets_sorted = sorted(retweets_pairing.items(), key = lambda kv: kv[1], reverse = True)
    
    # Return the sorted values:
    return retweets_sorted

Beide Funktionen können verschachtelt werden:

In [73]:
all_retweets = get_number_of_retweets(get_retweeted_accounts(data))

**Die Top 100 ist...**

In [75]:
top100 = all_retweets[:100]
top100

[('@realDonaldTrump', 918),
 ('@WhiteHouse', 655),
 ('@TeamTrump', 273),
 ('@GOPChairwoman', 211),
 ('@DanScavino', 145),
 ('@Jim_Jordan', 136),
 ('@GOP', 132),
 ('@Scavino45', 114),
 ('@DonaldJTrumpJr', 113),
 ('@IvankaTrump', 105),
 ('@GreggJarrett', 104),
 ('@EricTrump', 94),
 ('@TomFitton', 94),
 ('@paulsperry_', 85),
 ('@RepMarkMeadows', 80),
 ('@marklevinshow', 78),
 ('@TrumpWarRoom', 73),
 ('@LindseyGrahamSC', 72),
 ('@charliekirk11', 70),
 ('@dbongino', 69),
 ('@SenateGOP', 68),
 ('@GOPLeader', 66),
 ('@JudicialWatch', 59),
 ('@senatemajldr', 57),
 ('@foxandfriends', 56),
 ('@seanhannity', 55),
 ('@VP', 55),
 ('@SteveScalise', 53),
 ('@MZHemingway', 52),
 ('@FLOTUS', 52),
 ('@LouDobbs', 48),
 ('@DailyCaller', 46),
 ('@BuckSexton', 45),
 ('@RepAndyBiggsAZ', 45),
 ('@RepLeeZeldin', 45),
 ('@KimStrassel', 42),
 ('@Mike_Pence', 41),
 ('@MariaBartiromo', 41),
 ('@PressSec', 39),
 ('@SaraCarterDC', 38),
 ('@RepDougCollins', 38),
 ('@RepMattGaetz', 37),
 ('@NHC_Atlantic', 37),
 ('@Joh

<a href="#top">$\uparrow$ Top</a>